<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66


    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-22 14:43:04
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  3.11 L
Current:  1.42 C
-------------------
Today PnL: -1.45 L (-1.01%)
Current PnL: -18.24 L (-12.27%)
CY Booked + Current PnL: -6.80 L (-4.57%)
-------------------
Total profit:  1.70 L
Total loss:  -19.94 L
-------------------
Total Booked + Current PnL: 20.18 L (16.59%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.06%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.26 C
Est FTT PnL: 83.75 L (59.04%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.42%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,15.45,69.0,H-MC,3.51,199240.0,34570.0,8767.0,1.46,20.99,4.40,26.31,89.0,3.94,1.44,30.65,X40N,NTT,AC
76,TTKPRESTIG,770.00,99.36,45.0,M-SC,3.33,85936.0,-14841.0,14936.0,-1.25,-14.73,17.38,0.09,245.0,-0.99,0.62,12.90,OX40N,NTT,DURABLES
49,NATIONALUM,244.55,-43.66,64.0,H-MC,8.56,111798.0,11294.0,17809.0,-0.31,11.24,15.93,28.96,79.0,0.63,0.81,49.02,MH,ATH,METALS
41,ITC,452.00,-38.34,45.0,H-LC,2.29,198852.0,-1286.0,22172.0,-0.97,-0.64,11.15,10.44,4.0,-0.06,1.43,4.89,X40,NTT,FMCG
56,RELIANCE,1533.00,-13.29,48.0,H-LC,4.80,216840.0,6474.0,22313.0,-1.24,3.08,10.29,13.68,37.0,0.29,1.56,20.11,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-63.24,65.0,H-MC,3.90,261384.0,-40367.0,201788.0,6.52,-13.38,77.20,53.50,116.0,-0.20,1.88,16.10,XY24,NTT,FMCG
63,SHALBY,327.0,1126.48,71.0,M-SC,4.68,166715.0,-14428.0,59901.0,2.23,-7.97,35.93,25.10,235.0,-0.24,1.20,33.61,XY24,NTT,HEALTHCARE
80,VAIBHAVGBL,521.0,63.64,66.0,H-SC,6.51,141948.0,-40827.0,151899.0,1.76,-22.34,107.01,60.77,125.0,-0.27,1.02,27.14,XR,NTT,JEWELLERY
14,BLUESTARCO,2080.0,15.45,69.0,H-MC,3.51,199240.0,34570.0,8767.0,1.46,20.99,4.40,26.31,89.0,3.94,1.44,30.65,X40N,NTT,AC
34,ICICIPRULI,790.0,-19.73,51.0,H-MC,2.18,138063.0,2275.0,40480.0,1.39,1.68,29.32,31.48,107.0,0.06,1.00,14.00,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-4.62,49.0,H-LC,1.42,236452.0,-9699.0,88906.0,-4.63,-3.94,37.60,32.18,16.0,-0.11,1.70,30.68,X200,ATH,IT
53,RAJESHEXPO,518.0,1787.63,52.0,L-SC,2.45,51519.0,-85658.0,85753.0,-4.01,-62.44,166.45,0.07,267.0,-1.00,0.37,28.13,OX40N,NTT,JEWELLERY
26,GREENPANEL,537.0,222.35,54.0,M-SC,1.28,144033.0,-39045.0,118021.0,-3.42,-21.33,81.94,43.14,230.0,-0.33,1.04,35.48,XY24,NTT,MISC
83,VOLTAS,1530.0,-1.92,46.0,H-MC,2.73,205695.0,13953.0,23799.0,-3.42,7.28,11.57,19.69,99.0,0.59,1.48,14.57,XY25,NTT,AC
81,VALIANTORG,838.0,-318.96,48.0,H-SC,6.01,121771.0,-41834.0,163149.0,-3.36,-25.57,133.98,74.15,139.0,-0.26,0.88,56.09,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-30.39,50.0,H-SC,16.84,198773.0,-1181.0,98413.0,-1.71,-0.59,49.51,48.63,115.0,-0.01,1.43,10.89,XR,NTT,MISC
29,HAVELLS,2069.16,-0.37,55.0,H-MC,1.91,247026.0,-705.0,75763.0,-0.46,-0.28,30.67,30.30,92.0,-0.01,1.78,12.97,X40,ATH,ELECTRICAL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,99.36,45.0,M-SC,3.33,85936.0,-14841.0,14936.0,-1.25,-14.73,17.38,0.09,245.0,-0.99,0.62,12.90,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,91.37,49.0,H-SC,9.29,121068.0,-21411.0,40933.0,-2.07,-15.03,33.81,13.70,163.0,-0.52,0.87,42.89,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.87,64.0,H-SC,0.80,228150.0,-41517.0,77845.0,0.66,-15.40,34.12,13.47,138.0,-0.53,1.64,15.40,XY24,NTT,PAINTS
18,CERA,9475.0,-20.76,42.0,H-SC,2.16,144463.0,-31440.0,73459.0,0.27,-17.87,50.85,23.89,149.0,-0.43,1.04,24.76,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-21.69,45.0,H-MC,7.29,106048.0,-25787.0,68009.0,-2.46,-19.56,64.13,32.03,98.0,-0.38,0.76,18.78,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.0,15.45,69.0,H-MC,3.51,199240.0,34570.0,8767.0,1.46,20.99,4.4,26.31,89.0,3.94,1.44,30.65,X40N,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,48.36,70.0,M-MC,8.19,234123.0,9161.0,158501.0,-0.18,4.07,67.70,74.53,192.0,0.06,1.69,37.64,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,15.45,69.0,H-MC,3.51,199240.0,34570.0,8767.0,1.46,20.99,4.40,26.31,89.0,3.94,1.44,30.65,X40N,NTT,AC
17,CAMS,5211.76,-1.17,59.0,H-SC,3.11,112328.0,10324.0,33597.0,-2.17,10.12,29.91,43.06,122.0,0.31,0.81,28.97,X40N,ATH,MISC
83,VOLTAS,1530.00,-1.92,46.0,H-MC,2.73,205695.0,13953.0,23799.0,-3.42,7.28,11.57,19.69,99.0,0.59,1.48,14.57,XY25,NTT,AC
23,DABUR,735.00,-6.36,43.0,H-MC,6.97,253606.0,5538.0,102863.0,-2.34,2.23,40.56,43.70,102.0,0.05,1.83,17.00,XY24,BTT,FMCG


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,64.0,H-MC,8.56,111798.0,11294.0,17809.0,-0.31,11.24,15.93,28.96,79.0,0.63,0.81,49.02,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-41.04,49.0,M-SC,0.82,99668.0,8170.0,70585.0,-0.66,8.93,70.82,86.07,223.0,0.12,0.72,44.74,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,67.0,H-MC,6.60,185621.0,5813.0,106602.0,0.18,3.23,57.43,62.52,88.0,0.05,1.34,38.42,XR,NTT,BANKS
36,INDIAMART,4850.92,-53.52,33.0,H-SC,11.35,129961.0,6625.0,127141.0,-0.71,5.37,97.83,108.45,119.0,0.05,0.94,28.93,AR,ATH,MISC
85,WIPRO,420.00,-11.52,49.0,M-LC,5.68,155149.0,4204.0,105253.0,-2.20,2.78,67.84,72.51,53.0,0.04,1.12,8.98,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-17.78,38.0,H-LC,6.88,296128.0,-19714.0,132369.0,-2.17,-6.24,44.70,35.67,5.0,-0.15,2.13,6.75,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.08,41.0,H-LC,9.84,225844.0,-16080.0,79339.0,-1.97,-6.65,35.13,26.15,8.0,-0.20,1.63,6.72,X40,ATH,IT
73,TCS,4476.75,-27.72,44.0,H-LC,12.25,288364.0,-57592.0,132446.0,-3.20,-16.65,45.93,21.64,1.0,-0.43,2.08,2.15,X40,ATH,IT
5,ASIANPAINT,3465.66,-13.58,44.0,H-LC,7.72,217994.0,-33774.0,86980.0,-0.18,-13.41,39.90,21.13,27.0,-0.39,1.57,17.26,X40,ATH,PAINTS
41,ITC,452.00,-38.34,45.0,H-LC,2.29,198852.0,-1286.0,22172.0,-0.97,-0.64,11.15,10.44,4.0,-0.06,1.43,4.89,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-4.62,49.0,H-LC,1.42,236452.0,-9699.0,88906.0,-4.63,-3.94,37.60,32.18,16.0,-0.11,1.70,30.68,X200,ATH,IT
41,ITC,452.0,-38.34,45.0,H-LC,2.29,198852.0,-1286.0,22172.0,-0.97,-0.64,11.15,10.44,4.0,-0.06,1.43,4.89,X40,NTT,FMCG
64,SIEMENS,4671.5,0.81,55.0,H-LC,3.59,162150.0,-23945.0,71427.0,-2.49,-12.87,44.05,25.51,15.0,-0.34,1.17,19.49,AR,ATH,ELECTRICAL
50,NESTLEIND,1377.0,-9.75,50.0,H-LC,4.24,277758.0,12332.0,44469.0,-0.63,4.65,16.01,21.40,11.0,0.28,2.00,12.32,XY25,NTT,FMCG
56,RELIANCE,1533.0,-13.29,48.0,H-LC,4.80,216840.0,6474.0,22313.0,-1.24,3.08,10.29,13.68,37.0,0.29,1.56,20.11,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-27.72,44.0,H-LC,12.25,288364.0,-57592.0,132446.0,-3.20,-16.65,45.93,21.64,1.0,-0.43,2.08,2.15,X40,ATH,IT
41,ITC,452.00,-38.34,45.0,H-LC,2.29,198852.0,-1286.0,22172.0,-0.97,-0.64,11.15,10.44,4.0,-0.06,1.43,4.89,X40,NTT,FMCG
79,UNITDSPR,1644.00,-12.09,55.0,H-LC,7.67,230274.0,-3870.0,52502.0,0.80,-1.65,22.80,20.77,86.0,-0.07,1.66,5.15,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-1.08,41.0,H-LC,9.84,225844.0,-16080.0,79339.0,-1.97,-6.65,35.13,26.15,8.0,-0.20,1.63,6.72,X40,ATH,IT
82,VBL,671.64,-17.78,38.0,H-LC,6.88,296128.0,-19714.0,132369.0,-2.17,-6.24,44.70,35.67,5.0,-0.15,2.13,6.75,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,0.81,55.0,H-LC,3.59,162150.0,-23945.0,71427.0,-2.49,-12.87,44.05,25.51,15.0,-0.34,1.17,19.49,AR,ATH,ELECTRICAL
41,ITC,452.00,-38.34,45.0,H-LC,2.29,198852.0,-1286.0,22172.0,-0.97,-0.64,11.15,10.44,4.0,-0.06,1.43,4.89,X40,NTT,FMCG
19,CIPLA,1795.00,-20.47,47.0,H-LC,5.50,211994.0,7494.0,33919.0,-1.80,3.66,16.00,20.25,10.0,0.22,1.53,12.72,X40N,BTT,PHARMA
56,RELIANCE,1533.00,-13.29,48.0,H-LC,4.80,216840.0,6474.0,22313.0,-1.24,3.08,10.29,13.68,37.0,0.29,1.56,20.11,XY25,NTT,REFINERIES
5,ASIANPAINT,3465.66,-13.58,44.0,H-LC,7.72,217994.0,-33774.0,86980.0,-0.18,-13.41,39.90,21.13,27.0,-0.39,1.57,17.26,X40,ATH,PAINTS


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-27.72,44.0,H-LC,12.25,288364.0,-57592.0,132446.0,-3.20,-16.65,45.93,21.64,1.0,-0.43,2.08,2.15,X40,ATH,IT
39,INFY,2275.00,-18.52,48.0,H-LC,8.15,317775.0,4887.0,166800.0,-3.14,1.56,52.49,54.87,3.0,0.03,2.29,8.32,X40,BTT,IT
41,ITC,452.00,-38.34,45.0,H-LC,2.29,198852.0,-1286.0,22172.0,-0.97,-0.64,11.15,10.44,4.0,-0.06,1.43,4.89,X40,NTT,FMCG
82,VBL,671.64,-17.78,38.0,H-LC,6.88,296128.0,-19714.0,132369.0,-2.17,-6.24,44.70,35.67,5.0,-0.15,2.13,6.75,X40N,ATH,FMCG
1,ABB,7934.00,-38.50,58.0,H-LC,9.45,256920.0,-4699.0,123913.0,-1.55,-1.80,48.23,45.57,7.0,-0.04,1.85,9.32,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6950.00,61.0,L-SC,5.13,77327.0,-16483.0,93102.0,-1.26,-17.57,120.40,81.67,269.0,-0.18,0.56,49.75,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,49.0,L-SC,42.31,83021.0,-30528.0,70576.0,-2.44,-26.89,85.01,35.27,268.0,-0.43,0.60,103.20,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-173.86,66.0,M-SC,3.91,84810.0,-17400.0,110787.0,-1.30,-17.02,130.63,91.37,208.0,-0.16,0.61,37.67,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-5.85,43.0,H-SC,12.29,87582.0,-13381.0,107910.0,-1.98,-13.25,123.21,93.63,148.0,-0.12,0.63,30.45,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,52.0,H-SC,12.62,94500.0,-3480.0,25080.0,-0.11,-3.55,26.54,22.05,152.0,-0.14,0.68,37.36,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1126.48,71.0,M-SC,4.68,166715.0,-14428.0,59901.0,2.23,-7.97,35.93,25.10,235.0,-0.24,1.20,33.61,XY24,NTT,HEALTHCARE
70,TANLA,1963.11,-15.81,71.0,H-SC,13.41,225216.0,-45545.0,375503.0,-2.35,-16.82,166.73,121.86,133.0,-0.12,1.62,76.18,AR,ATH,IT
60,SAMMAANCAP,326.00,-173.86,66.0,M-SC,3.91,84810.0,-17400.0,110787.0,-1.30,-17.02,130.63,91.37,208.0,-0.16,0.61,37.67,XY25,NTT,FINANCE
80,VAIBHAVGBL,521.00,63.64,66.0,H-SC,6.51,141948.0,-40827.0,151899.0,1.76,-22.34,107.01,60.77,125.0,-0.27,1.02,27.14,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3721.62,55.0,M-SC,4.97,119074.0,-28294.0,80113.0,-1.86,-19.20,67.28,35.16,236.0,-0.35,0.86,21.62,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.62
1,25,44.23
2,50,75.26


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.78
LC    31.13
MC    24.08
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.87
X40      14.99
X40N     11.66
XR       11.48
AR        8.57
XY25      8.39
OX40N     7.97
X200      1.70
MH        1.68
X5K       1.44
SR        1.24
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.95
H-LC    24.55
H-MC    21.09
M-SC    13.30
M-LC     5.53
M-MC     2.68
L-SC     1.53
L-LC     1.05
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.14,-15.20,78.01
FMCG,12.43,-4.01,41.22
FINANCE,10.24,-15.59,66.02
MISC,7.21,-14.97,78.88
BANKS,6.20,-16.01,77.04
PAINTS,5.75,-15.44,32.64
ELECTRICAL,5.60,-5.95,43.13
INSURANCE,3.77,-5.94,45.55
AC,3.63,3.86,12.23


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3097255.0,22
XR,1316820.0,14
AR,1143833.0,9
X40,769380.0,10
X40N,588693.0,9
OX40N,540015.0,10
XY25,382388.0,6
SR,248659.0,2
X5K,118280.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3636691.0,29
M-SC,1399343.0,17
H-MC,1181323.0,15
H-LC,1120515.0,14
M-LC,383567.0,4
M-MC,305741.0,2
L-SC,249431.0,3
L-MC,60171.0,1
L-LC,38321.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1218483.0      6
M-SC       XY24       809125.0      7
H-SC       AR         784262.0      5
           XR         767032.0      7
H-MC       XY24       540912.0      4
H-LC       X40        487737.0      5
M-MC       XY24       305741.0      2
H-SC       X40N       297540.0      4
           OX40N      257650.0      4
           SR         248659.0      2
H-LC       X40N       218790.0      3
H-MC       X40        209130.0      4
H-LC       AR         195340.0      2
H-MC       XY25       166498.0      2
M-SC       AR         164231.0      2
L-SC       XR         163678.0      2
M-LC       XY24       160034.0      2
M-SC       OX40N      128603.0      4
M-LC       X5K        118280.0      1
M-SC       XR         114084.0      2
           XY25       110787.0      1
H-MC       XR         106602.0      1
M-LC       XR         105253.0      1
H-LC       X200        88906.0      1
L-SC       OX40N       85753.0      1
M-SC       X40         72513.0      1
H-MC       X40N        72363.0      2
           OX40N       68009.0      1
H-LC       XY25        66782.0      2
H-SC       MH          63065.0      1
H-LC       XY24        62960.0      1
L-MC       XR          60171.0      1
L-LC       XY25        38321.0      1
H-MC       MH          17809.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
